In [1]:
import heapq
import datetime
import sys
from collections import deque

class AlgorithmRequirments:
    def __init__(self, grid, parent=None, move=None, depth=0, cost=0):
        self.grid = grid
        self.parent = parent
        self.move = move
        self.depth = depth
        self.cost = cost
        self.heuristic = 0
    def title_move_place(self):
        for i in range(3):
            for j in range(3):
                if self.grid[i][j] == 0:
                    return i, j

    def title_actions(self):
        successors = []
        blank_row, blank_col = self.title_move_place()
        moves = [
            ('Up', blank_row - 1, blank_col),
            ('Down', blank_row + 1, blank_col),
            ('Left', blank_row, blank_col - 1),
            ('Right', blank_row, blank_col + 1)
        ]
        for move, row, col in moves:
            if 0 <= row < 3 and 0 <= col < 3:
                new_grid = [r[:] for r in self.grid]
                new_grid[blank_row][blank_col], new_grid[row][col] = new_grid[row][col], new_grid[blank_row][blank_col]
                move_cost = new_grid[blank_row][blank_col]
                successors.append(AlgorithmRequirments(new_grid, self, move, self.depth + 1, self.cost + move_cost))
        return successors
    def __lt__(self, other):
        return (self.cost + self.heuristic) < (other.cost + other.heuristic)

    def __eq__(self, other):
        return self.grid == other.grid

    def __repr__(self):
        return f"< state = {self.grid}, action = {{{self.move}}}, g(n) = {self.cost}, d = {self.depth}, f(n) = {self.cost + self.heuristic}, Parent = {self.parent.grid if self.parent else 'None'} >"

    def __hash__(self):
        return hash(str(self.grid))

    def calculate_heuristic(self, goal_state):
        goal_list = [value for row in goal_state.grid for value in row]
        distance = 0

        for i in range(3):
            for j in range(3):
                value = self.grid[i][j]
                if value != 0:
                    goal_index = goal_list.index(value)
                    goal_row, goal_col = divmod(goal_index, 3)
                    distance += abs(goal_row - i) + abs(goal_col - j)
        return distance

In [2]:
def matrix(file_path):
    values = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.strip() == "END OF FILE":
                break
            values.append(list(map(int, line.split())))
    return values

Breadth First Search

In [3]:

def bfs(start_state, goal_state, trace_file):
    visited = set()
    fifo = deque([start_state])
    popped = 0
    expanded = 0
    generated = 0
    fringe_size = 0

    with open(trace_file, 'w') as f:
        f.write(f"Method Selected: BFS\nRunning BFS\n")

        while fifo:
            max_fringe_size = max(fringe_size, len(fifo))
            current_state = fifo.popleft()
            popped += 1
            expanded += 1

            if current_state.grid == goal_state.grid:
                return current_state, popped, expanded, generated, max_fringe_size

            visited.add(current_state)
            successors = current_state.title_actions()
            generated += len(successors)

            f.write(f"Generating successors to < state = {current_state.grid}, action = {{{current_state.move or 'Start'}}}, g(n) = {current_state.cost}, d = {current_state.depth}, Parent = Pointer to {{{current_state.parent}}} >:\n")
            f.write(f"\t{len(successors)} successors generated\n")

            for successor in successors:
                if successor not in visited:
                    fifo.append(successor)

            f.write(f"\tClosed: {list(map(lambda x: x.grid, visited))}\n")
            f.write(f"\tFringe: {list(map(lambda x: f'< state = {x.grid}, action = {{{x.move}}}, g(n) = {x.cost}, d = {x.depth}, Parent = Pointer to {{{x.parent}}} >', fifo))}\n")

    return None, popped,expanded, generated, max_fringe_size


Uniform Cost Search

In [4]:

def ucs(start_state, goal_state, trace_file):
    visited = set()
    fifo = []
    start_state.cost = 0
    heapq.heappush(fifo, (start_state.cost, start_state))

    popped = 0
    expanded = 0
    generated = 0
    fringe_size = 0

    with open(trace_file, 'w') as f:
        f.write(f"Method Selected: UCS\nRunning UCS\n")

        while fifo:
            max_fringe_size = max(fringe_size, len(fifo))
            _, current_state = heapq.heappop(fifo)
            popped += 1
            expanded += 1

            if current_state.grid == goal_state.grid:
                return current_state, popped, expanded, generated, max_fringe_size

            visited.add(current_state)
            successors = current_state.title_actions()
            generated += len(successors)

            f.write(f"Generating successors to < state = {current_state.grid}, action = {{{current_state.move or 'Start'}}}, g(n) = {current_state.cost}, d = {current_state.depth}, Parent = Pointer to {{{current_state.parent}}} >:\n")
            f.write(f"\t{len(successors)} successors generated\n")

            for successor in successors:
                if successor not in visited:
                    heapq.heappush(fifo, (successor.cost, successor))

            f.write(f"\tClosed: {list(map(lambda x: x.grid, visited))}\n")
            f.write(f"\tFringe: {list(map(lambda x: f'< state = {x[1].grid}, action = {{{x[1].move}}}, g(n) = {x[1].cost}, d = {x[1].depth}, Parent = Pointer to {{{x[1].parent}}} >', fifo))}\n")

    return None, popped, expanded, generated, max_fringe_size


Depth First Search

In [5]:

def dfs(start_state, goal_state, trace_file):
    visited = set()
    lifo = [start_state]
    popped = 0
    expanded = 0
    generated = 0
    fringe_size = 0

    with open(trace_file, 'w') as f:
        f.write(f"Method Selected: DFS\nRunning DFS\n")

        while lifo:
            max_fringe_size = max(fringe_size, len(lifo))
            current_state = lifo.pop()
            popped += 1
            expanded += 1

            if current_state.grid == goal_state.grid:
                return current_state, popped, expanded,generated, max_fringe_size

            visited.add(current_state)
            successors = current_state.title_actions()
            generated += len(successors)

            f.write(f"Generating successors to < state = {current_state.grid}, action = {{{current_state.move or 'Start'}}}, g(n) = {current_state.cost}, d = {current_state.depth}, Parent = Pointer to {{{current_state.parent}}} >:\n")
            f.write(f"\t{len(successors)} successors generated\n")

            for successor in successors:
                if successor not in visited:
                    lifo.append(successor)

            f.write(f"\tClosed: {list(map(lambda x: x.grid, visited))}\n")
            f.write(f"\tFringe: {list(map(lambda x: f'< state = {x.grid}, action = {{{x.move}}}, g(n) = {x.cost}, d = {x.depth}, Parent = Pointer to {{{x.parent}}} >', lifo))}\n")

    return None, popped, expanded, generated, max_fringe_size


Depth Limit Search

In [6]:
def dls(start_state, goal_state, depth_limit, trace_file):
    visited = set()
    lifo = [(start_state, 0)]
    popped = 0
    expanded = 0
    generated = 0
    fringe_size = 0

    with open(trace_file, 'w') as f:
        f.write(f"Method Selected: DLS\nRunning DLS with depth limit {depth_limit}\n")

        while lifo:
            max_fringe_size = max(fringe_size, len(lifo))
            current_state, current_depth = lifo.pop()
            popped += 1
            expanded += 1

            if current_state.grid == goal_state.grid:
                return current_state,popped, expanded, generated, max_fringe_size

            if current_depth < depth_limit:
                visited.add(current_state)
                successors = current_state.title_actions()
                generated += len(successors)

                f.write(f"Generating successors to < state = {current_state.grid}, action = {{{current_state.move or 'Start'}}}, g(n) = {current_state.cost}, d = {current_state.depth}, Parent = Pointer to {{{current_state.parent}}} >:\n")
                f.write(f"\t{len(successors)} successors generated\n")

                for successor in successors:
                    if successor not in visited:
                        lifo.append((successor, current_depth + 1))

                f.write(f"\tClosed: {list(map(lambda x: x.grid, visited))}\n")
                f.write(f"\tFringe: {list(map(lambda x: f'< state = {x[0].grid}, action = {{{x[0].move}}}, g(n) = {x[0].cost}, d = {x[0].depth}, Parent = Pointer to {{{x[0].parent}}} >', lifo))}\n")

    return None, popped, expanded,generated, max_fringe_size


Iterative Deepening Search

In [7]:
def ids(start_state, goal_state, trace_file):
    def level(state, depth, trace_file):
        visited = set()
        lifo = [(state, 0)]
        popped = 0
        expanded = 0
        generated = 0
        fringe_size = 0

        with open(trace_file, 'a') as f:
            f.write(f"Running DFS with depth limit {depth}\n")

            while lifo:
                max_fringe_size = max(fringe_size, len(lifo))
                current_state, current_depth = lifo.pop()
                popped += 1
                expanded += 1

                if current_state.grid == goal_state.grid:
                    return current_state, popped, expanded, generated, max_fringe_size

                if current_depth < depth:
                    visited.add(current_state)
                    successors = current_state.title_actions()
                    generated += len(successors)

                    f.write(f"Generating successors to < state = {current_state.grid}, action = {{{current_state.move or 'Start'}}}, g(n) = {current_state.cost}, d = {current_state.depth}, Parent = Pointer to {{{current_state.parent}}} >:\n")
                    f.write(f"\t{len(successors)} successors generated\n")

                    for successor in successors:
                        if successor not in visited:
                            lifo.append((successor, current_depth + 1))

                    f.write(f"\tClosed: {list(map(lambda x: x.grid, visited))}\n")
                    f.write(f"\tFringe: {list(map(lambda x: f'< state = {x[0].grid}, action = {{{x[0].move}}}, g(n) = {x[0].cost}, d = {x[0].depth}, Parent = Pointer to {{{x[0].parent}}} >', lifo))}\n")

        return None, popped, expanded, generated, max_fringe_size

    depth = 0
    with open(trace_file, 'w') as f:
        f.write(f"Method Selected: IDS\nRunning IDS\n")

    while True:
        result, popped, expanded, generated, max_fringe_size = level(start_state, depth, trace_file)
        if result:
            return result, popped, expanded, generated, max_fringe_size
        depth += 1


Greedy Algorithm

In [8]:
def greedy(start_state, goal_state, trace_file):
    visited = set()
    costAdded = []
    start_state.heuristic_value = start_state.calculate_heuristic(goal_state)
    heapq.heappush(costAdded, (start_state.heuristic_value, start_state))

    popped = 0
    expanded = 0
    generated = 0
    fringe_size = 0

    with open(trace_file, 'w') as f:
        f.write(f"Method Selected: Greedy\nRunning Greedy\n")

        while costAdded:
            max_fringe_size = max(fringe_size, len(costAdded))
            _, current_state = heapq.heappop(costAdded)
            popped += 1
            expanded += 1

            if current_state.grid == goal_state.grid:
                return current_state, popped, expanded, generated, max_fringe_size

            visited.add(current_state)
            successors = current_state.title_actions()
            generated += len(successors)

            f.write(f"Generating successors to < state = {current_state.grid}, action = {{{current_state.move or 'Start'}}}, g(n) = {current_state.cost}, d = {current_state.depth}, f(n) = {current_state.cost + current_state.heuristic_value}, Parent = Pointer to {{{current_state.parent}}} >:\n")
            f.write(f"\t{len(successors)} successors generated\n")

            for successor in successors:
                if successor not in visited:
                    successor.heuristic_value = successor.calculate_heuristic(goal_state)
                    heapq.heappush(costAdded, (successor.heuristic_value, successor))

            f.write(f"\tClosed: {list(map(lambda x: x.grid, visited))}\n")
            f.write(f"\tFringe: {list(map(lambda x: f'< state = {x[1].grid}, action = {{{x[1].move}}}, g(n) = {x[1].cost}, d = {x[1].depth}, f(n) = {x[1].cost + x[1].heuristic_value}, Parent = Pointer to {{{x[1].parent}}} >', costAdded))}\n")

    return None, popped, expanded, generated, max_fringe_size


A-Star

In [9]:
def a_star(start_state, goal_state, trace_file):
    visited = set()
    costAdded = []
    start_state.heuristic_value = start_state.calculate_heuristic(goal_state)
    heapq.heappush(costAdded, (start_state.cost + start_state.heuristic_value, start_state))

    popped = 0
    expanded = 0
    generated = 0
    fringe_size = 0

    with open(trace_file, 'w') as f:
        f.write(f"Method Selected: A*\nRunning A*\n")

        while costAdded:
            max_fringe_size = max(fringe_size, len(costAdded))
            _, current_state = heapq.heappop(costAdded)
            popped += 1
            expanded += 1

            if current_state.grid == goal_state.grid:
                return current_state, popped, expanded,generated, max_fringe_size

            visited.add(current_state)
            successors = current_state.title_actions()
            generated += len(successors)

            f.write(f"Generating successors to < state = {current_state.grid}, action = {{{current_state.move or 'Start'}}}, g(n) = {current_state.cost}, d = {current_state.depth}, f(n) = {current_state.cost + current_state.heuristic_value}, Parent = Pointer to {{{current_state.parent}}} >:\n")
            f.write(f"\t{len(successors)} successors generated\n")

            for successor in successors:
                if successor not in visited:
                    successor.heuristic_value = successor.calculate_heuristic(goal_state)
                    heapq.heappush(costAdded, (successor.cost + successor.heuristic_value, successor))

            f.write(f"\tClosed: {list(map(lambda x: x.grid, visited))}\n")
            f.write(f"\tFringe: {list(map(lambda x: f'< state = {x[1].grid}, action = {{{x[1].move}}}, g(n) = {x[1].cost}, d = {x[1].depth}, f(n) = {x[1].cost + x[1].heuristic_value}, Parent = Pointer to {{{x[1].parent}}} >', costAdded))}\n")

    return None, popped,expanded, generated, max_fringe_size


Output Block

In [15]:
def display_result(goal_state, nodes_popped, nodes_generated, max_fringe_size, nodes_expanded=None):
    solution_steps = []
    current_state = goal_state

    while current_state.parent:
        move = current_state.grid[current_state.parent.title_move_place()[0]][current_state.parent.title_move_place()[1]]
        solution_steps.append(f"Move {move} {current_state.move}")
        current_state = current_state.parent

    solution_steps.reverse()

    print(f"Nodes Popped: {nodes_popped}")
    if nodes_expanded is not None:
        print(f"Nodes Expanded: {nodes_expanded}")
    print(f"Nodes Generated: {nodes_generated}")
    print(f"Max Fringe Size: {max_fringe_size}")
    print(f"Solution Found at depth {goal_state.depth} with cost of {goal_state.cost}.")
    print("Steps:")
    for step in solution_steps:
        print(f"\t{step}")

def main():
    start_file = 'start.txt'
    goal_file = 'goal.txt'
    method = input("Enter the search method (bfs, ucs, dfs, dls, ids, greedy, a-star): ").strip()
    trace_file = f'trace_{method}_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}.txt'
    start_state = AlgorithmRequirments(matrix(start_file))
    goal_state = AlgorithmRequirments(matrix(goal_file))
    if not method:
       method = "a-star"

    print(f"Selected algorithm: {method}")
    if method == "bfs":
        result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size = bfs(start_state, goal_state, trace_file)
    elif method == "ucs":
        result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size = ucs(start_state, goal_state, trace_file)
    elif method == "dfs":
        result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size = dfs(start_state, goal_state, trace_file)
    elif method == "dls":
        depth_limit = int(input("Enter depth limit: "))
        result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size = dls(start_state, goal_state, depth_limit, trace_file)
    elif method == "ids":
        result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size = ids(start_state, goal_state, trace_file)
    elif method == "greedy":
        result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size = greedy(start_state, goal_state, trace_file)
    else:
        result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size = a_star(start_state, goal_state, trace_file)

    if result:
        display_result(result, nodes_popped, nodes_expanded, nodes_generated, max_fringe_size)
    else:
        print("No solution found.")

if __name__ == "__main__":
    main()



Enter the search method (bfs, ucs, dfs, dls, ids, greedy, a-star): dls
Selected algorithm: dls
Enter depth limit: 10
Nodes Popped: 170
Nodes Expanded: 8
Nodes Generated: 170
Max Fringe Size: 271
Solution Found at depth 8 with cost of 49.
Steps:
	Move 5 Left
	Move 6 Up
	Move 8 Right
	Move 5 Down
	Move 6 Left
	Move 8 Up
	Move 5 Right
	Move 6 Down
